# HW1: LeNet-5 with Post-training Quantization and Quantization Aware Training
[LeNet](http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf) is considered to be the first ConvNet.
We are going to implement a neural architecture similar to LeNet and train it with [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset.

Before we start, you may check this [Tensorspace-LeNet](https://tensorspace.org/html/playground/lenet.html) to play with LeNet and get familiar with this neural architecture.

![image](https://production-media.paperswithcode.com/methods/LeNet_Original_Image_48T74Lc.jpg)
Ref.: LeCun et al., Gradient-Based Learning Applied to Document Recognition, 1998a

<font color='red'>Name: (Replace with your name) Student ID: (Replace with your Student ID) </font>

## 1. Initial Setup

We are goint to implement and train this nerual network with PyTorch 
If you are not familer with PyTorch, check [official tutorail](https://pytorch.org/tutorials/beginner/basics/intro.html)

**Reminder:** set the runtime type to "GPU", or your code will run much more slowly on a CPU.

In [ ]:
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torch import optim
from torch.utils.data import DataLoader
import numpy as np
import random
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

### 1.1 Load dataset
Load training and test data from the MNIST dataset.

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

transform = transforms.Compose(
    [
     transforms.Resize((32, 32)),
     transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))
    ])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2,
                                          worker_init_fn=seed_worker, generator=g,)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=8,
                                         shuffle=False, num_workers=2,
                                         worker_init_fn=seed_worker, generator=g,)

### 1.2 Define the Neural Network 
Define a simple CNN that classifies MNIST images.

In [ ]:
from collections import OrderedDict
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(OrderedDict([
            ('conv', nn.Conv2d(1, 6, 5, bias=False)),
            ('relu', nn.ReLU()),
        ]))
        
        self.maxpool2 = nn.Sequential(OrderedDict([
            ('pool', nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        ]))
        
        self.conv3 = nn.Sequential(OrderedDict([
            ('conv', nn.Conv2d(6, 16, 5, bias=False)),
            ('relu', nn.ReLU())
        ]))
        
        self.maxpool4 = nn.Sequential(OrderedDict([
            ('pool', nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        ]))
        
        self.conv5 = nn.Sequential(OrderedDict([
            ('conv', nn.Conv2d(16, 120, 5, bias=False)),
            ('relu', nn.ReLU())
        ]))
        
        self.fc6 = nn.Sequential(OrderedDict([
            ('fc', nn.Linear(120, 84, bias=False)),
            ('relu', nn.ReLU())
        ]))
        
        self.output = nn.Sequential(OrderedDict([
            ('fc', nn.Linear(84, 10, bias=False)),
        ]))
        

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool4(x)
        x = self.conv5(x)
        x = torch.flatten(x, 1)
        x = self.fc6(x)
        x = self.output(x)
        return x

NET = Net().to(device)

### 1.3 Question: Profile the Neural Architecture by TorchInfo
Torchinfo provides information complementary to what is provided by print(your_model) in PyTorch, similar to Tensorflow's model.summary() API to view the visualization of the model, which is helpful while debugging your network. Check this [link](https://github.com/TylerYep/torchinfo#how-to-use) about how to use TorchInfo by `summary()` and fill in the TODO in the following cell. You should get the result similar to the table below:

```
==========================================================================================
Layer (type:depth-idx)                   Output Shape              Param #
==========================================================================================
LeNet                                    --                        --
...
...
==========================================================================================
Total params: ...
...
Estimated Total Size (MB): ...
==========================================================================================
```


Ref.: https://github.com/TylerYep/torchinfo

Please read *B. LeNet-5* in the [original paper](http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf) and answer the following questions.
1. What is the type (convolution, pooling, fully-connected layer, etc.), input activation size, output activation size, and activation function (if any) of each layer?
2. What is the difference part between this neural architecture and the lenet-5 in the [original paper](http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf)?
3. Could we replace the 3rd conv, the conv in conv5, with a fully connected layer?

### 1.3 Answers
<font color='red'>Write your answers here.</font>
1. ...
2. ...
3. ...

In [ ]:
from torchinfo import summary
# TODO


### 1.4 Train and Test the Neural Network
Train this CNN on the training dataset (this may take a few moments).
* Check how to save and load the model
    * https://pytorch.org/tutorials/beginner/saving_loading_models.html
    * Save:
        ```
        torch.save(model.state_dict(), PATH)
        ```
    * Load:
        ```
        model = TheModelClass(*args, **kwargs)
        model.load_state_dict(torch.load(PATH))
        model.eval()
        ```
* After training the model, we will save it as `lenet.pt`.
* You should comment out `train(NET, trainloader, 2)` and uncomment `NET.load_state_dict(torch.load('lenet.pt'))` before submitting your homework.
    * Also, reloading the model from `lenet.pt` can save your time if there is something wrong and you need to restart and run all.

In [ ]:
def train(model: nn.Module, dataloader: DataLoader, num_epoch):
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epoch):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(dataloader):
            
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
        print(test(model, testloader, None))
    print('Finished Training')
    
def test(model: nn.Module, dataloader: DataLoader, max_samples=None, device=device) -> float:
    correct = 0
    total = 0
    n_inferences = 0

    with torch.no_grad():
        for data in dataloader:
            
            inputs, labels = data[0].to(device), data[1].to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if max_samples:
                n_inferences += inputs.shape[0]
                if n_inferences > max_samples:
                    break
    
    return 100 * correct / total

In [ ]:
train(NET, trainloader, 2)
# NET.load_state_dict(torch.load('lenet.pt'))
score = test(NET, testloader, None)
print('Accuracy of the network on the test images: {}%'.format(score))

torch.save(NET.state_dict(), 'lenet.pt')

In [ ]:
from copy import deepcopy

# A convenience function which we use to copy CNNs
def copy_model(model: nn.Module) -> nn.Module:
    result = deepcopy(model)

    # Copy over the extra metadata we've collected which copy.deepcopy doesn't capture
    if hasattr(model, 'input_activations'):
        result.input_activations = deepcopy(model.input_activations)

    for result_layer, original_layer in zip(result.modules(), model.modules()):
        if isinstance(result_layer, (nn.Conv2d, nn.Linear)):
            if hasattr(original_layer.weight, 'scale'):
                result_layer.weight.scale = deepcopy(original_layer.weight.scale)
            
        if hasattr(original_layer, 'inAct'):
            result_layer.inAct = deepcopy(original_layer.inAct)
        if hasattr(original_layer, 'outAct'):
            result_layer.outAct = deepcopy(original_layer.outAct)
        if hasattr(original_layer, 'output_scale'):
            result_layer.output_scale = deepcopy(original_layer.output_scale)

    return result

## 2. Post-training Quantization
### 2.1 Question: Visualize Weights

We have flattened all vector for you by `tensor.view(-1)`.

Try plotting a histogram of each weight and show the total range and 3-sigma range for each weight.

hint: `np.histogram()` and `plt.hist()`

In [ ]:
import matplotlib.pyplot as plt

conv1_weights = NET.conv1[0].weight.data.cpu().view(-1)
conv2_weights = NET.conv3[0].weight.data.cpu().view(-1)
conv3_weights = NET.conv5[0].weight.data.cpu().view(-1)
fc1_weights = NET.fc6[0].weight.data.cpu().view(-1)
fc2_weights = NET.output[0].weight.data.cpu().view(-1)

weightDict = {
    'conv1_weights':conv1_weights, 
    'conv2_weights': conv2_weights, 
    'conv3_weights': conv3_weights, 
    'fc1_weights': fc1_weights, 
    'fc2_weights':fc2_weights
}

# TODO


### 2.2 Question:  Quantize Weights
Computation of convolution or fully-connected layer can be expressed as
$$W\times I = O$$
where $W$ is the weight tensor, $I$ is the input tensor, and $O$ is the output tensor.
Let $n_w$ be the scaling factor. We have $$W_q\times I =n_w W \times I\approx n_w O$$ where $W_q$ is the quantized 8-bit signed integer weight tensor.

Fill in the TODO in `quantized_weights()`.If you’ve done everything correctly, the accuracy degradation should be negligible(~1%).
1. What is $n_w$? Explain how you get it.
2. What is the accuracy degradation? \
Show both relative error and absolute error when the true value is the accuracy we get before performing any quanitzion.

### 2.2 Answers
<font color='red'>Write your answers here.</font>
1. ...
2. ...

In [ ]:
net_q2 = copy_model(NET)

In [ ]:
from typing import Tuple

def quantized_weights(weights: torch.Tensor) -> Tuple[torch.Tensor, float]:
    '''
    Quantize the weights so that all values are integers between -128 and 127.
    Use the total range when deciding just what factors to scale the float32 
    values by.

    Parameters:
    weights (Tensor): The unquantized weights

    Returns:
    (Tensor, float): A tuple with the following elements:
        * The weights in quantized form, where every value is an integer between -128 and 127.
          The "dtype" will still be "float", but the values themselves should all be integers.
        * The scaling factor that your weights were multiplied by.
          This value does not need to be an 8-bit integer.
    '''

    # TODO
    # Adopt the symmetric quantization by the total range
    
  
    
    return weights, 1

In [ ]:
def quantize_layer_weights(model: nn.Module):
    for layer in model.modules():
        if isinstance(layer, (nn.Conv2d, nn.Linear)):
            q_layer_data, scale = quantized_weights(layer.weight.data)
            q_layer_data = q_layer_data.to(device)

            layer.weight.data = q_layer_data
            layer.weight.scale = scale

            if (q_layer_data < -128).any() or (q_layer_data > 127).any():
                raise Exception("Quantized weights of {} layer include values out of bounds for an 8-bit signed integer".format(layer.__class__.__name__))
            if (q_layer_data != q_layer_data.round()).any():
                raise Exception("Quantized weights of {} layer include non-integer values".format(layer.__class__.__name__))

quantize_layer_weights(net_q2)

In [ ]:
score = test(net_q2, testloader)
print('Accuracy of the network after quantizing all weights: {}%'.format(score))

### 2.3 Question: Visualize Activations
Plot histograms of the input images and the output activations of each operation and answer the following questions.
1. Discuss any observations about the distribution of these activations.
2. Record the range of the values, as well as their 3-sigma range (the difference between μ + 3σ and μ − 3σ).

### 2.3 Answers
<font color='red'>Write your answers here.</font>
1. ...
2. ...

In [ ]:
net_q3 = copy_model(NET)

def visualize_activations(module, input, output):
    if module.profile_activations is True:
        module.inAct = input[0].cpu().reshape(-1)
        module.outAct = output[0].cpu().reshape(-1)
    
for name, model in net_q3.named_children():
    print("{}\n [register_forward_hook]: {}".format(name, model))
    model.profile_activations = True
    model.register_forward_hook(visualize_activations)
net_q3.eval()
with torch.no_grad():
    input = trainset[0][0].unsqueeze(0)
    _ = net_q3(input.to(device))   
for name, model in net_q3.named_children(): 
    model.profile_activations = False 

In [ ]:
input_activations = net_q3.conv1.inAct
conv1_output_activations = net_q3.conv1.outAct
conv3_output_activations = net_q3.conv3.outAct
conv5_output_activations = net_q3.conv5.outAct
fc6_output_activations = net_q3.fc6.outAct
output_output_activations = net_q3.output.outAct

actDict = {
    'input_activations':input_activations, 
    'conv1_output_activations':conv1_output_activations, 
    'conv3_output_activations':conv3_output_activations, 
    'conv5_output_activations':conv5_output_activations, 
    'fc6_output_activations':fc6_output_activations, 
    'output_output_activations':output_output_activations
}

# TODO


### 2.4 Question:  Quantize Activations
The output of conv in conv1 can be $$W_{conv1}\times I=O_{conv1}.$$
Let the scaling factor of the quantized input matrix $I$ be $n_I$, the scaling factor of the quantized weight matrix $W_{conv1}$ 
be $n_{W_{conv1}}$, and the scaling factor of the output matrix $O_{conv1}$ be $n_{O_{conv1}}$.  
$$W_{conv1_q}\times I_q = (n_{W_{conv1}}W_{conv1})\times (n_II)\approx (n_{W_{conv1}}n_I)O_{conv1}$$
where $W_{conv1_q}$ is the quantized 8-bit signed integer weight tensor and $I_q$ is the quantized 8-bit signed integer input activation tensor.

$$O_{conv1_q} \approx n_{O_{conv1}}O_{conv1}$$
where $O_{conv1_q}$ is the quantized 8-bit signed integer output activation tensor.

Since we're doing post-training quantization, we can get $n_I$, $n_{W_{conv1}}$, and $n_{O_{conv1}}$ first and do the other calculations for quantization.

As for `forward()` of `NetQuantized()`, make sure you can simulate fixed-point representation when doing any calculation with input/output scale. Keep in mind that we will implement hardware to accelerate this model with fixed-point computations.
   * In this assignment, we only "emulate" fixed-point computations. We don't need to use any fixed-point data type (e.g., `int`).
   * You will have to fill in the TODO in `forward()` to scale the outputs of each layer. Consider rounding binary fractions to the 16th place with the following steps (e.g., for output_scale): 
- Initial input:
   To scaling the initial input, you can simply perform `round(scale)*features` since `input_scale` is much greater than 1.
- the convolution/fully connected layer:
       1. `scale = round(scale*(2**16))`: Now, we have the `scale` rounded to the 16th place with a software trick of moving the binary point (`*(2**16)`) and applying the round function.
       2. emulating bit-shifting in fixed-point numbers instead of floating-point numbers by either of the following:
        1.	`floor(input × scale >> 16)`
        2.	`int(input × scale >> 16)`
       4. Clamp the value between -128 and 127
    
    

Answer the following questions.

1. How to compute $n_I$, $n_{W_{conv1}}$, and $n_{O_{conv1}}$? 
2. The ture quantized output activation tensor is depend on $W_{conv1_q}$ and $I_q$, so we cannot simply apply only $n_{O_{conv1}}$ on the output of $W_{conv1_q}\times I_q$ to quantize output activation. \
Derive an equation for the quantized output of the conv in conv1 after quantizing activation and weight with  $n_I$, $n_{W_{conv1}}$, and $n_{O_{conv1}}$ and show the scaling factor $S_1$ of it. \
(hint: re-quantize $O'$ in $W_{conv1_q}\times I_q = O'$.That is to have $S_1(W_{conv1_q}\times I_q) \approx S_1O' = O_{conv1_q}$ where $O_{conv1_q}$ is the quantized 8-bit signed integer output.)
3. Derive an equation for the quantized output of the conv in conv3 after quantizing activation and weight.
4. Show the general equantion of each layer for calculating the scaling factor of output activation.
5. Fill in the TODO in the following code.If you’ve done everything correctly, the accuracy degradation should be negligible(~1%). What is the accuracy degradation? Show both relative error and absolute error when the true value is the accuracy we get before performing any quanitzion.

### 2.4 Answers
<font color='red'>Write your answers here.</font>
1. ...
2. ...
3. ...
4. ...
5. ...


In [ ]:
from typing import List

class NetQuantized(nn.Module):
    def __init__(self, net_with_weights_quantized: nn.Module):
        super(NetQuantized, self).__init__()
        
        net_init = copy_model(net_with_weights_quantized)

        self.conv1 = net_init.conv1
        self.maxpool2 = net_init.maxpool2
        self.conv3 = net_init.conv3
        self.maxpool4 = net_init.maxpool4
        self.conv5 = net_init.conv5
        self.fc6 = net_init.fc6
        self.output = net_init.output

        for layer in self.conv1, self.conv3, self.conv5, self.fc6, self.output:
            def pre_hook(l, x):
                x = x[0]
                if (x < -128).any() or (x > 127).any():
                    raise Exception("Input to {} layer is out of bounds for an 8-bit signed integer".format(l.__class__.__name__))
                if (x != x.round()).any():
                    raise Exception("Input to {} layer has non-integer values".format(l.__class__.__name__))
            layer.register_forward_pre_hook(pre_hook)

        # Calculate the scaling factor for the initial input to the CNN
        self.input_activations = net_with_weights_quantized.conv1.inAct
        self.input_scale = NetQuantized.quantize_initial_input(self.input_activations)

        # Calculate the output scaling factors for all the layers of the CNN
        preceding_layer_scales = []
        for layer in self.conv1, self.conv3, self.conv5, self.fc6, self.output:
            layer.output_scale = NetQuantized.quantize_activations(layer.outAct, layer[0].weight.scale, self.input_scale, preceding_layer_scales)
            preceding_layer_scales.append((layer[0].weight.scale, layer.output_scale))

    @staticmethod
    def quantize_initial_input(pixels: np.ndarray) -> float:
        '''
        Calculate a scaling factor for the images that are input to the first layer of the CNN.

        Parameters:
        pixels (ndarray): The values of all the pixels which were part of the input image during training

        Returns:
        float: A scaling factor that the input should be multiplied by before being fed into the first layer.
               This value does not need to be an 8-bit integer.
        '''

        # TODO
        
        
        return 1

    @staticmethod
    def quantize_activations(activations: np.ndarray, n_w: float, n_initial_input: float, n_s: List[Tuple[float, float]]) -> float:
        '''
        Calculate a scaling factor to multiply the output of a layer by.

        Parameters:
        activations (ndarray): The values of all the pixels which have been output by this layer during training
        n_w (float): The scale by which the weights of this layer were multiplied as part of the "quantize_weights" function you wrote earlier
        n_initial_input (float): The scale by which the initial input to the neural network was multiplied
        n_s ([(float, float)]): A list of tuples, where each tuple represents the "weight scale" and "output scale" (in that order) for every preceding layer

        Returns:
        float: A scaling factor that the layer output should be multiplied by before being fed into the first layer.
               This value does not need to be an 8-bit integer.
        '''
        # TODO

        
        return 1

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        '''
        Since input_scale is 128 and all output_scales are less than 1, we should keep input_scale as it is and tranform output_scale to 
        round(1/output_scale) to ease the verilog implementaion.
        
        Also, the normalized input images is a matrix with lots of floating numbers. We can transform x*input_scale to
        input_scale/round(1/x)
        
        To not implement rounding in verilog, we use floor when doing other calculations with input/output_scale.
        '''
        # To make sure that the outputs of each layer are integers between -128 and 127, you may need to use the following functions:
        #   * torch.Tensor.round
        #   * torch.Tensor.floor
        #   * torch.clamp

        # TODO
        
                
        return x

In [ ]:
net_init = copy_model(net_q2)
net_init.input_activations = deepcopy(net_q3.conv1.inAct)
        
for layer_init, layer_q3 in zip(net_init.children(), net_q3.children()):
    layer_init.inAct = deepcopy(layer_q3.inAct)
    layer_init.outAct = deepcopy(layer_q3.outAct)

net_quantized = NetQuantized(net_init)

In [ ]:
score = test(net_quantized, testloader)
print('Accuracy of the network after quantizing both weights and activations: {}%'.format(score))

Answer the following questions.(hint: please consider verilog implementation):

6. What is the benefit of using `floor`?
7. What is the benefit of replacing `x*output_scale`with `x/round(1/output_scale)`?

### 2.4 Answers
<font color='red'>Write your answers here.</font>
6. ...
7. ...

In [ ]:
print("input_scale:\n", net_quantized.input_scale.item())
print("output_scale:\n {}\n {}\n {}\n {}\n {}".format(
    net_quantized.conv1.output_scale.item(),
    net_quantized.conv3.output_scale.item(),
    net_quantized.conv5.output_scale.item(),
    net_quantized.fc6.output_scale.item(),
    net_quantized.output.output_scale.item()
))

print("input_scale:\n", net_quantized.input_scale.item())
print("output_scale:\n {}\n {}\n {}\n {}\n {}".format(
    round(1/net_quantized.conv1.output_scale.item()),
    round(1/net_quantized.conv3.output_scale.item()),
    round(1/net_quantized.conv5.output_scale.item()),
    round(1/net_quantized.fc6.output_scale.item()),
    round(1/net_quantized.output.output_scale.item())
))

### 2.5 Question:  Quantize Biases
We add a bias in the final layer of this LeNet.
* You should comment out `train(NET_WITH_BIAS, trainloader, 2)` and uncomment `NET_WITH_BIAS.load_state_dict(torch.load('lenet_with_bias.pt'))` before submitting your homework.
    * Also, reloading the model from `lenet_with_bias.pt` can save your time if there is something wrong and you need to restart and run all.

Answer the following questions.
1. Now the equation is $$W*I+\beta = O$$ where $\beta$ is the bias. Derive the equation of a quantized layer with bias.\
Note that our biases are commonly quantized to 32-bits
2. What is the scaling factor for the bias?\
(hint: the form looks just like what we have done for quantizing activations)
3. Fill in the TODO in the following code.If you’ve done everything correctly, the accuracy degradation should be negligible(~1%). What is the accuracy degradation?\
Show both relative error and absolute error when the true value is the accuracy we get before performing any quanitzion.

### 2.5 Answers
<font color='red'>Write your answers here.</font>
1. ...
2. ...
3. ...

In [ ]:
class NetWithBias(Net):
    def __init__(self):
        super(NetWithBias, self).__init__()

        self.output = nn.Sequential(OrderedDict([
            ('fc', nn.Linear(84, 10, bias=True)),
        ]))
    
NET_WITH_BIAS = NetWithBias().to(device)

In [ ]:
train(NET_WITH_BIAS, trainloader, 2)
# NET_WITH_BIAS.load_state_dict(torch.load('lenet_with_bias.pt'))
score = test(NET_WITH_BIAS, testloader)
print('Accuracy of the network (with a bias) on the test images: {}%'.format(score))
torch.save(NET_WITH_BIAS.state_dict(), 'lenet_with_bias.pt')

In [ ]:
for name, model in NET_WITH_BIAS.named_children():
    print("{}\n [register_forward_hook]: {}".format(name, model))
    model.profile_activations = True
    model.register_forward_hook(visualize_activations)
NET_WITH_BIAS.eval()
with torch.no_grad():
    input = trainset[0][0].unsqueeze(0)
    _ = NET_WITH_BIAS(input.to(device))
for name, model in NET_WITH_BIAS.named_children(): model.profile_activations = False 

In [ ]:
net_with_bias_with_quantized_weights = copy_model(NET_WITH_BIAS)
quantize_layer_weights(net_with_bias_with_quantized_weights)

score = test(net_with_bias_with_quantized_weights, testloader)
print('Accuracy of the network on the test images after all the weights are quantized but the bias isn\'t: {}%'.format(score))

In [ ]:
class NetQuantizedWithBias(NetQuantized):
    def __init__(self, net_with_weights_quantized: nn.Module):
        super(NetQuantizedWithBias, self).__init__(net_with_weights_quantized)
        preceding_scales = [
            (self.conv1[0].weight.scale, self.conv1.output_scale),
            (self.conv3[0].weight.scale, self.conv3.output_scale),
            (self.conv5[0].weight.scale, self.conv5.output_scale),
            (self.fc6[0].weight.scale, self.fc6.output_scale),
            (self.output[0].weight.scale, self.output.output_scale)
        ][:-1]
        
        self.output[0].bias.data = NetQuantizedWithBias.quantized_bias(
            self.output[0].bias.data,
            self.output[0].weight.scale,
            self.input_scale,
            preceding_scales
        )

        if (self.output[0].bias.data < -2147483648).any() or (self.output[0].bias.data > 2147483647).any():
            raise Exception("Bias has values which are out of bounds for an 32-bit signed integer")
        if (self.output[0].bias.data != self.output[0].bias.data.round()).any():
            raise Exception("Bias has non-integer values")

    @staticmethod
    def quantized_bias(bias: torch.Tensor, n_w: float, n_initial_input: float, n_s: List[Tuple[float, float]]) -> torch.Tensor:
        '''
        Quantize the bias so that all values are integers between -2147483648 and 2147483647.

        Parameters:
        bias (Tensor): The floating point values of the bias
        n_w (float): The scale by which the weights of this layer were multiplied
        n_initial_input (float): The scale by which the initial input to the neural network was multiplied
        n_s ([(float, float)]): A list of tuples, where each tuple represents the "weight scale" and "output scale" (in that order) for every preceding layer

        Returns:
        Tensor: The bias in quantized form, where every value is an integer between -2147483648 and 2147483647.
                The "dtype" will still be "float", but the values themselves should all be integers.
        '''

        # TODO
        # --------------------------------------------------------------------
        # The quantization way are similar to quantize output activation
        # --------------------------------------------------------------------

        
        return torch.clamp((bias).round(), min=-2147483648, max=2147483647)

In [ ]:
net_quantized_with_bias = NetQuantizedWithBias(net_with_bias_with_quantized_weights)

In [ ]:
score = test(net_quantized_with_bias, testloader)
print('Accuracy of the network on the test images after all the weights and the bias are quantized: {}%'.format(score))

## 3. Quantization Aware Training (QAT)
* You should comment out `train(model_fp32_prepared, trainloader, 2)` and uncomment `model_fp32_prepared.load_state_dict(torch.load('model_fp32_prepared.pt'))` before submitting your homework.
    * Also, reloading the model from `model_fp32_prepared.pt` can save your time if there is something wrong and you need to restart and run all.

### 3.1 Question: 
Try to trace code and study the quantization-aware training (QAT) from [Quantization — PyTorch 1.13 documentation](https://pytorch.org/docs/stable/quantization.html), then answer the following question.

1. How can the QAT achieve a higher accuracy than the post-training quantization (PTQ)?
2. What does the function model_fp32_fused do to improve precision and speed?
3. Two more layers (quant, dequant) appeared after we quantized our model using the PyTorch QAT method. What do these two layers do?

### 3.1 Answers
<font color='red'>Write your answers here.</font>
1. ...
2. ...
3. ...

In [ ]:
class QATNet(NetWithBias):
    def __init__(self):
        super().__init__()
        # QuantStub converts tensors from floating point to quantized
        self.quant = torch.quantization.QuantStub()
        # DeQuantStub converts tensors from quantized to floating point
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = super().forward(x)
        x = self.dequant(x)
        return x
    
MODEL_FP32 = QATNet().to(device)
print(MODEL_FP32)

MODEL_FP32.eval()
print(MODEL_FP32)

MODEL_FP32.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
MODEL_FP32_FUSED = torch.quantization.fuse_modules(MODEL_FP32,
    [['conv1.conv', 'conv1.relu'], 
     ['conv3.conv', 'conv3.relu'],
     ['conv5.conv', 'conv5.relu'],
     ['fc6.fc', 'fc6.relu'],
    ])
MODEL_FP32_FUSED = MODEL_FP32
# Specify quantization configuration
MODEL_FP32_FUSED.qconfig = torch.ao.quantization.get_default_qat_qconfig('fbgemm')
print(MODEL_FP32_FUSED.qconfig)

model_fp32_prepared = torch.quantization.prepare_qat(MODEL_FP32_FUSED.train())

train(model_fp32_prepared, trainloader, 2)
# model_fp32_prepared.load_state_dict(torch.load('model_fp32_prepared.pt'))
score = test(model_fp32_prepared, testloader)
print('Accuracy of the model_fp32_prepared: {}%'.format(score))
torch.save(model_fp32_prepared.state_dict(), 'model_fp32_prepared.pt')

In [ ]:
# model_fp32_prepared

In [ ]:
model_fp32_prepared = model_fp32_prepared.to('cpu')
model_fp32_prepared.eval()
qat = torch.quantization.convert(model_fp32_prepared, inplace=False)


# Extract the input and output of the quantized model

In [ ]:
# copy the model with bias quantized model and save the weights
inference_model = copy_model(qat)

In [ ]:
# Use accuray to make sure it is same model
score = test(inference_model, testloader, None, torch.device('cpu'))
print('Accuracy of the network after quantizing both weights and activations: {}%'.format(score))

Use an image as an input of the activations，and choose 100 images to generate patterns for our homework 2.

In [ ]:
# random choose images as the input and get the output
np.random.seed(0)
index = np.random.randint(0,len(trainset), size=100)
index = range(100)

Save the activations of input and output to the CSV format.

In [ ]:
import os 
import zipfile
# It is easier to download all the files with zip
zf = zipfile.ZipFile('parameters.zip', 'w', zipfile.ZIP_DEFLATED)

if not os.path.exists('./activations'):
    os.mkdir('./activations')

In [ ]:
for ind in range(100):
    if not os.path.exists('./activations/img{}'.format(ind)):
        os.mkdir('./activations/img{}'.format(ind))

    for name, model in qat.named_children():
        model.profile_activations = True
        model.register_forward_hook(visualize_activations)
    input0, label = testset[index[ind]]
    input = input0.reshape(1, 1, 32, 32)
    output = qat(input)
    for name, model in qat.named_children(): model.profile_activations = False 
    

    np.savetxt('./activations/img{}/input.csv'.format(ind), input.cpu().data.numpy().reshape(-1), delimiter=',')
    np.savetxt('./activations/img{}/output.csv'.format(ind), output.cpu().data.numpy().reshape(-1).astype(int), delimiter=',')
    zf.write('./activations/img{}/input.csv'.format(ind))
    zf.write('./activations/img{}/output.csv'.format(ind))
    
    opDict = {
        'conv1': (qat.conv1.inAct, qat.conv1.outAct),
        'maxpool2': (qat.maxpool2.inAct, qat.maxpool2.outAct),
        'conv3': (qat.conv3.inAct, qat.conv3.outAct),
        'maxpool4': (qat.maxpool4.inAct, qat.maxpool4.outAct),
        'conv5': (qat.conv5.inAct, qat.conv5.outAct),
        'fc6': (qat.fc6.inAct, qat.fc6.outAct),
        'quant': (qat.quant.inAct, qat.quant.outAct),
        'dequant': (qat.dequant.inAct, qat.dequant.outAct),
        'output': (qat.output.inAct, qat.output.outAct)
    }
    
    for key in opDict:
        if not os.path.exists('./activations/img{}/{}'.format(ind, key)):
            os.mkdir('./activations/img{}/{}'.format(ind, key))
        if(opDict[key][0].type()== "torch.quantized.QInt8Tensor" or opDict[key][0].type()== "torch.quantized.QUInt8Tensor"):
            temp = opDict[key][0].cpu().int_repr()
        else:
            temp = opDict[key][0].cpu()
        if(opDict[key][1].type()== "torch.quantized.QInt8Tensor" or opDict[key][1].type()== "torch.quantized.QUInt8Tensor"):
            temp1 = opDict[key][1].cpu().int_repr()
        else:
            temp1 = opDict[key][1].cpu()                
        np.savetxt('./activations/img{}/{}/input.csv'.format(ind, key), temp.data.numpy().reshape(-1).astype(float), delimiter=',')
        np.savetxt('./activations/img{}/{}/output.csv'.format(ind, key), temp1.cpu().data.numpy().reshape(-1).astype(float), delimiter=',')
        zf.write('./activations/img{}/{}/input.csv'.format(ind, key))
        zf.write('./activations/img{}/{}/output.csv'.format(ind, key))

Save each layer's weights, zero point and scaling factor to the CSV format.

In [ ]:
if not os.path.exists('./weights'):
    os.mkdir('./weights')
if not os.path.exists('./weights/scale'):
    os.mkdir('./weights/scale')
if not os.path.exists('./weights/weight'):
    os.mkdir('./weights/weight')
if not os.path.exists('./weights/zero_point'):
    os.mkdir('./weights/zero_point')
for name, weights in qat.state_dict().items():
    name_split = name.split('.')
    if weights!= None:
        if name_split[-2] != "_packed_params":
            if(weights.type()== "torch.quantized.QInt8Tensor" or weights.type()== "torch.quantized.QUInt8Tensor"):
                np.savetxt('./weights/weight/%s.csv' %(name) , weights.cpu().int_repr().numpy().reshape(-1).astype(float), delimiter=',')
                zf.write('./weights/weight/%s.csv' %(name))

                np.savetxt('./weights/weight/%s.scale.csv' %(name) , weights.q_per_channel_scales().numpy().reshape(-1).astype(float), delimiter=',')
                zf.write('./weights/weight/%s.scale.csv' %(name))

                np.savetxt('./weights/weight/%s.zero_point.csv' %(name) , weights.q_per_channel_zero_points().cpu().numpy().reshape(-1).astype(float), delimiter=',')
                zf.write('./weights/weight/%s.zero_point.csv' %(name))
            else:
                np.savetxt('./weights/%s/%s.csv'%(name_split[-1],name) , weights.cpu().numpy().reshape(-1).astype(float), delimiter=',')
                zf.write('./weights/%s/%s.csv'%(name_split[-1],name))

          
          
        elif name_split[-1] == "_packed_params":
            if not os.path.exists('./weights/_packed_params'):
                os.mkdir('./weights/_packed_params')
            weight, bias = weights
            name = name_split[0]+"."+name_split[1]
            if(weight.type()== "torch.quantized.QInt8Tensor" or weight.type()== "torch.quantized.QUInt8Tensor"):
                np.savetxt('./weights/weight/%s.weight.csv' %(name)  , weight.cpu().int_repr().numpy().reshape(-1).astype(float), delimiter=',')
                zf.write('./weights/weight/%s.weight.csv' %(name))

                np.savetxt('./weights/weight/%s.weight.scale.csv' %(name) , weight.q_per_channel_scales().cpu().numpy().reshape(-1).astype(float), delimiter=',')
                zf.write('./weights/weight/%s.weight.scale.csv' %(name))

                np.savetxt('./weights/weight/%s.weight.zero_point.csv' %(name) , weight.q_per_channel_zero_points().cpu().numpy().reshape(-1).astype(float), delimiter=',')
                zf.write('./weights/weight/%s.weight.zero_point.csv' %(name))
            
            if bias is not None:
                
                if(bias.type()== "torch.quantized.QInt8Tensor" or bias.type()== "torch.quantized.QUInt8Tensor"):
                    np.savetxt('./weights/weight/%s.bias.csv' %(name) , bias.cpu().int_repr().numpy().reshape(-1).astype(float), delimiter=',')
                    zf.write('./weights/weight/%s.bias.csv' %(name))
                else:
                    np.savetxt('./weights/weight/%s.bias.csv' %(name) , bias.cpu().detach().numpy().reshape(-1).astype(float), delimiter=',')
                    zf.write('./weights/weight/%s.bias.csv' %(name))


Save the zip file.

In [ ]:
zf.close()